# Answering Business Questions using SQL

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Overview of the Data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ('table','view');

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM customer
LIMIT 5;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [4]:
%%sql
SELECT *
FROM employee
LIMIT 5;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


## Selecting New Albums to Purchase

In [5]:
%%sql
WITH usa AS
        (
        SELECT *
        FROM customer
        WHERE country = 'USA'
        ),
     usa_track AS
        (
        SELECT t.track_id,
               g.name
        FROM usa
        INNER JOIN invoice i ON usa.customer_id = i.customer_id
        INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
        INNER JOIN track t ON il.track_id = t.track_id
        INNER JOIN genre g ON t.genre_id = g.genre_id
        ),
     track_sold AS
        (
        SELECT name AS genre_name, 
               COUNT(*) AS number_of_tracks
        FROM usa_track
        GROUP BY 1
        ORDER BY 1)
        
SELECT genre_name,
       number_of_tracks,
       ROUND((CAST(number_of_tracks AS FLOAT) / 1051) * 100, 2) AS percentage_of_tracks
FROM track_sold
ORDER BY 3 DESC;

 * sqlite:///chinook.db
Done.


genre_name,number_of_tracks,percentage_of_tracks
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


From the table above, we can see the top genres in USA. From what artists we have available to add to our stores, we have to add three.

We should add Artists:
1. *Red Tone* with Genre *Punk* which is having 12.37% share,
2. *Slim Jim Bites* with Genre *Blues* which is having 3.43% share,
3. *Meteor and the Girls* with Genre *Pop* which is having 2.09% share.

## Analyzing Employee Sales Performance

In [6]:
%%sql
WITH agent AS
        (
        SELECT *
        FROM employee
        WHERE title = 'Sales Support Agent'
        ),
     sales AS
        (
        SELECT *
        FROM agent
        INNER JOIN customer c ON agent.employee_id = c.support_rep_id
        ),
     performance AS
        (
        SELECT *
        FROM sales
        INNER JOIN invoice i ON sales.customer_id = i.customer_id
        )
SELECT employee_id AS agent_id,
       first_name || ' ' || last_name AS agent_name,
       SUM(total) AS total_sales,
       hire_date
FROM performance
GROUP BY 1
ORDER BY 1;

 * sqlite:///chinook.db
Done.


agent_id,agent_name,total_sales,hire_date
3,Jane Peacock,1731.5100000000039,2017-04-01 00:00:00
4,Margaret Park,1584.0000000000034,2017-05-03 00:00:00
5,Steve Johnson,1393.920000000002,2017-10-17 00:00:00


Above table tells us that *Jane Peacock* has the *most* sales and *Steve Johnson* has the *least* but important factor in this is related to their *hiring dates*.

## Analyzing Sales by Country

In [7]:
%%sql
WITH country_group AS
                (
                SELECT
                       country, 
                       COUNT(*) AS num_of_customers
                FROM customer
                GROUP BY 1
                ),
     invoice_group AS
                (
                SELECT 
                       billing_country,
                       COUNT(*) orders,
                       SUM(total) AS total
                FROM invoice
                GROUP BY 1
                ),
     country_invoice_group AS
                (
                SELECT 
                       CASE
                            WHEN num_of_customers = 1 THEN 'Other'
                            ELSE country
                       END AS country,
                       num_of_customers,
                       orders, 
                       total
                FROM country_group AS c
                INNER JOIN invoice_group AS i ON c.country = i.billing_country
                ),
     final_table AS
                (
                SELECT 
                       country,
                       SUM(num_of_customers) AS num_of_customers,
                       SUM(orders) AS orders,
                       SUM(total) AS total,
                       CASE
                           WHEN country = 'Other' THEN 1
                           ELSE 0
                       END AS sort
                FROM country_invoice_group
                GROUP BY 1
                ORDER BY sort ASC
                )
SELECT country,
       num_of_customers AS customers,
       total AS total_sales,
       total / num_of_customers AS average_sales,
       total / orders AS average_order_value
FROM final_table
ORDER BY sort ASC, total_sales DESC;

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_sales,average_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9399999999998,72.996,7.4485714285714275


Based on the data, there may be opportunity in the following countries:

- Czech Republic
- United Kingdom
- India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low.  Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence.  A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Albums vs Individual Tracks

In [8]:
%%sql
SELECT il.invoice_id, t.track_id, a.album_id
FROM invoice_line il
INNER JOIN track t ON il.track_id = t.track_id
INNER JOIN album a ON t.album_id = a.album_id;

 * sqlite:///chinook.db
Done.


invoice_id,track_id,album_id
1,1158,91
1,1159,91
1,1160,91
1,1161,91
1,1162,91
1,1163,91
1,1164,91
1,1165,91
1,1166,91
1,1167,91


In [9]:
%%sql
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) 
                                                FROM invoice
                                      ) AS percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases.  Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.

## Artist in Playlist

In [10]:
%%sql
SELECT 
        ar.name AS 'Artist Name', 
        COUNT(pt.playlist_id) AS 'Number of Playlists'
FROM playlist_track pt
INNER JOIN track t ON pt.track_id = t.track_id
INNER JOIN album a ON t.album_id = a.album_id
INNER JOIN artist ar ON a.artist_id = ar.artist_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10; 

 * sqlite:///chinook.db
Done.


Artist Name,Number of Playlists
Iron Maiden,516
U2,333
Metallica,296
Led Zeppelin,252
Deep Purple,226
Lost,184
Pearl Jam,177
Eric Clapton,145
Faith No More,145
Lenny Kravitz,143


We can see that `Iron Maiden` is the artist that has been used in most of the playlists.

## Tracks Purchased vs Tracks Not Purchased

In [29]:
%%sql
SELECT 
        COUNT(DISTINCT(il.track_id)) AS 'Tracks Purchased', 
        COUNT(DISTINCT(t.track_id)) - COUNT(DISTINCT(il.track_id)) AS 'Tracks Not Puchased'
FROM track t
LEFT JOIN invoice_line il ON t.track_id = il.track_id;

 * sqlite:///chinook.db
Done.


Tracks Purchased,Tracks Not Puchased
1806,1697


We can see that out of `3503` tracks `1806` which is approx. `51.5%` have been purchased.